In [120]:
with open("data/TinyStoriesV2-GPT4-valid.txt", 'r') as f:
    corpus = f.read()[:3000]
print(corpus)

u don't have to be scared of the loud dog, I'll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.
<|endoftext|>
Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.
Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."
Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."
They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could hear t

In [ ]:
import regex as re

def strip_of_special_tokens(corpus, special_tokens):
    """Strips of special tokens to avoid counting them as bytes"""

    # Escape | delimiter in special tokens
    for i in range(len(special_tokens)):
        if "|" in special_tokens[i]:
            special_tokens[i] = re.escape(special_tokens[i])

    # Join special tokens into a delim for a splitting pattern
    delim = "|".join(special_tokens)
    chunks = re.split(delim, corpus)
    # Remove empty chunks
    chunks = [ch for ch in chunks if ch.strip()]
    return chunks

special_tokens = ["<|endoftext|>", "<start>", "<end>"]
corpus = strip_of_special_tokens(corpus, special_tokens)
corpus

['u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\nTom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\nSam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\nThey went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could hear 

In [ ]:
vocab_size = 262
num_of_merges = vocab_size - 256
vocab = {}

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

# Pre-tokenization
def pretokenize(corpus, ptrn):
    """Pre-tokenizes on regex pattern"""

    counts = {}
    for t in corpus:
        for word in re.findall(ptrn, t):
            counts[word] = counts.get(word, 0) + 1
    return counts

corpus = pretokenize(corpus, PAT)
corpus

{'u': 1,
 ' don': 1,
 "'t": 1,
 ' have': 3,
 ' to': 21,
 ' be': 1,
 ' scared': 5,
 ' of': 6,
 ' the': 28,
 ' loud': 2,
 ' dog': 1,
 ',': 25,
 ' I': 5,
 "'ll": 1,
 ' protect': 1,
 ' you': 5,
 '".': 1,
 ' The': 2,
 ' mole': 3,
 ' felt': 4,
 ' so': 2,
 ' safe': 3,
 ' with': 7,
 ' little': 2,
 ' girl': 1,
 '.': 68,
 ' She': 8,
 ' was': 13,
 ' very': 3,
 ' kind': 1,
 ' and': 30,
 ' soon': 1,
 ' came': 2,
 ' trust': 1,
 ' her': 4,
 ' He': 8,
 ' leaned': 1,
 ' against': 1,
 ' she': 2,
 ' kept': 1,
 ' him': 7,
 ' had': 2,
 ' found': 1,
 ' his': 7,
 ' best': 1,
 ' friend': 2,
 '\n': 20,
 'Once': 1,
 ' upon': 1,
 ' a': 11,
 ' time': 1,
 ' in': 5,
 ' warm': 1,
 ' sunny': 1,
 ' place': 1,
 ' there': 1,
 ' big': 2,
 ' pit': 9,
 ' A': 1,
 ' boy': 1,
 ' named': 1,
 ' Tom': 11,
 ' liked': 3,
 ' play': 1,
 ' near': 1,
 ' One': 1,
 ' day': 1,
 ' lost': 1,
 ' red': 2,
 ' ball': 7,
 ' sad': 3,
 'Tom': 4,
 ' asked': 3,
 ' Sam': 2,
 ' help': 2,
 ' search': 3,
 ' for': 6,
 ' They': 16,
 ' looked': 3,
 ' high

In [ ]:
def split_to_bytes(corpus):
    """Splits words by characters and counts frequency"""

    counts = {}
    # Count byte pairs
    for k, v in corpus.items():
        new_key = tuple([c for c in k])
        counts[new_key] = v

    # Sort by the highest frequency
    counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
    return counts

corpus = split_to_bytes(corpus)
corpus

{('.',): 68,
 (' ', 'a', 'n', 'd'): 30,
 (' ', 't', 'h', 'e'): 28,
 (',',): 25,
 (' ', 't', 'o'): 21,
 ('\n',): 20,
 (' ', 'T', 'h', 'e', 'y'): 16,
 (' ', 'w', 'a', 's'): 13,
 (' ', 'a'): 11,
 (' ', 'T', 'o', 'm'): 11,
 (' ', 'p', 'i', 't'): 9,
 (' ', 'w', 'e', 'r', 'e'): 9,
 (' ', 'S', 'h', 'e'): 8,
 (' ', 'H', 'e'): 8,
 (' ', 'w', 'i', 't', 'h'): 7,
 (' ', 'h', 'i', 'm'): 7,
 (' ', 'h', 'i', 's'): 7,
 (' ', 'b', 'a', 'l', 'l'): 7,
 (' ', 't', 'h', 'e', 'i', 'r'): 7,
 (' ', 'b', 'l', 'o', 'c', 'k', 's'): 7,
 ('"',): 7,
 (' ', 'o', 'f'): 6,
 (' ', 'f', 'o', 'r'): 6,
 (' ', 'b', 'u', 't'): 6,
 (' ', 't', 'h', 'e', 'y'): 6,
 (' ', 'n', 'o', 't'): 6,
 (' ', 's', 'a', 'i', 'd'): 6,
 (' ', 's', 'c', 'a', 'r', 'e', 'd'): 5,
 (' ', 'I'): 5,
 (' ', 'y', 'o', 'u'): 5,
 (' ', 'i', 'n'): 5,
 (' ', 'L', 'i', 'l', 'y'): 5,
 (' ', 'a', 'r', 'e'): 5,
 (' ', 'f', 'e', 'l', 't'): 4,
 (' ', 'h', 'e', 'r'): 4,
 ('T', 'o', 'm'): 4,
 (' ', 'f', 'i', 'n', 'd'): 4,
 (' ', 'w', 'a', 'n', 't', 'e', 'd'): 4,
 (

In [ ]:
from collections import defaultdict

def count_bytepairs(corpus, counts=None, mf_pair=None, merged_words=None):
    """Counts bytepair frequencies in the corpus
    If ran the first time (no counts provided) then count all byte pairs in the whole corpus
    If ran consecutively, remove the most frequent pair count as it's merged now
    and count byte pairs only on the merged words now"""

    bp_to_counts = {}
    bp_to_words = defaultdict(set)

    # If counts are provided, remove the most frequent byte pair from the counts and then re-count only the merged words
    if counts:
        # Remove the most frequent pair first
        counts.pop(mf_pair)

        # Count only merged words as only these are updated
        for w in merged_words:
           for c1, c2 in zip(w, w[1:]):
                bp_to_counts[(c1, c2)] = bp_to_counts.get((c1, c2), 0) + corpus[w]
                bp_to_words[(c1, c2)].add(w) 
    
    # For the first time we need to count every single pair
    else:
        # Count every single pair in the corpus
        for k, v in corpus.items():
            for c1, c2 in zip(k, k[1:]):
                bp_to_counts[(c1, c2)] = bp_to_counts.get((c1, c2), 0) + v
                bp_to_words[(c1, c2)].add(k)

    return bp_to_counts, bp_to_words

In [125]:
counts, counts_to_words = count_bytepairs(corpus)
counts

{(' ', 'a'): 56,
 ('a', 'n'): 48,
 ('n', 'd'): 38,
 (' ', 't'): 93,
 ('t', 'h'): 63,
 ('h', 'e'): 93,
 ('t', 'o'): 40,
 (' ', 'T'): 30,
 ('T', 'h'): 21,
 ('e', 'y'): 24,
 (' ', 'w'): 43,
 ('w', 'a'): 20,
 ('a', 's'): 17,
 ('T', 'o'): 15,
 ('o', 'm'): 25,
 (' ', 'p'): 20,
 ('p', 'i'): 9,
 ('i', 't'): 22,
 ('w', 'e'): 17,
 ('e', 'r'): 32,
 ('r', 'e'): 32,
 (' ', 'S'): 10,
 ('S', 'h'): 8,
 (' ', 'H'): 8,
 ('H', 'e'): 8,
 ('w', 'i'): 7,
 (' ', 'h'): 39,
 ('h', 'i'): 18,
 ('i', 'm'): 10,
 ('i', 's'): 13,
 (' ', 'b'): 33,
 ('b', 'a'): 8,
 ('a', 'l'): 12,
 ('l', 'l'): 18,
 ('e', 'i'): 8,
 ('i', 'r'): 8,
 ('b', 'l'): 8,
 ('l', 'o'): 20,
 ('o', 'c'): 8,
 ('c', 'k'): 11,
 ('k', 's'): 7,
 (' ', 'o'): 15,
 ('o', 'f'): 6,
 (' ', 'f'): 23,
 ('f', 'o'): 8,
 ('o', 'r'): 16,
 ('b', 'u'): 8,
 ('u', 't'): 11,
 (' ', 'n'): 11,
 ('n', 'o'): 7,
 ('o', 't'): 9,
 (' ', 's'): 48,
 ('s', 'a'): 13,
 ('a', 'i'): 8,
 ('i', 'd'): 17,
 ('s', 'c'): 7,
 ('c', 'a'): 17,
 ('a', 'r'): 32,
 ('e', 'd'): 41,
 (' ', 'I'): 9,

In [ ]:
def get_mf_pair(counts, counts_to_words):
    """Takes the most frequent byte pair along with words with that pair and returns them"""

    # Get the max frequency
    maxf = counts[max(counts, key=counts.get)]

    # Get the candidates with the max frequency
    candidates = [k for k, v in counts.items() if v == maxf]
    # Pick the lexicographically greater pair
    pair = max(candidates)
    return pair, counts_to_words[pair]

In [127]:
mf_pair, mf_pair_words = get_mf_pair(counts, counts_to_words)
mf_pair

('h', 'e')

In [ ]:
def merge(corpus, merge_pair, merge_pair_words):
    """Merges the word in the corpus 
    by joining two bytes into one and
    re-assigning the key in the dictionary"""

    # Keeps track of merged words to update byte counting only on these as these only change
    merged_words = set()
    
    # Merging the keys 
    for w in merge_pair_words:
        new_k = []
        b = 0
        while b < len(w):
            if b + 1 < len(w):
                c1, c2 = w[b], w[b+1]
                if c1 + c2 == merge_pair:
                    new_k.append(merge_pair)
                    b += 2
                else:
                    new_k.append(c1)
                    b += 1
            else:
                new_k.append(w[b])
                b += 1
        # Add new merged word
        corpus[tuple(new_k)] = corpus[w]

        # Add to merged words to optimize byte pair counts as this only changes and the rest is still the same
        merged_words.add(tuple(new_k))
    
    # Pop the unmerged words
    for w in merge_pair_words:
        corpus.pop(w)
    
    return corpus, merged_words

In [79]:
def pair_to_bytes(pair):
    return tuple(b.encode("utf-8") for b in pair)

In [80]:
merges = []

# Start merges
for i in range(num_of_merges):
    # Count bytepairs
    counts = count_bytepairs(corpus)
    # Get the most frequent pair
    pair = get_mf_pair(counts)
    # Add merge to merges
    pair_b = pair_to_bytes(pair)
    merges.append(pair_b)
    # Add the merge to the vocab
    merge_b = "".join(pair).encode("utf-8")
    vocab[256 + i] = merge_b
    # Apply the merge to the corpus
    corpus = merge(corpus, "".join(pair))

In [81]:
vocab

{256: b'he', 257: b' t', 258: b' a', 259: b' s', 260: b' the', 261: b' w'}

In [82]:
merges

[(b'h', b'e'),
 (b' ', b't'),
 (b' ', b'a'),
 (b' ', b's'),
 (b' t', b'he'),
 (b' ', b'w')]